In [67]:
import pandas as pd
import numpy as np
import scipy.linalg as sla
import scipy.sparse.linalg as ssla
#import pdb
import copy
from datetime import datetime
from timeit import default_timer as timer

In [68]:
data = pd.read_csv('characteristics_data_feb2017.csv', index_col=0, low_memory=False)
data = data.query("yy >= 1990")
data.sort_values(['date', 'permno'], inplace=True)

In [69]:
# Convert to dictionary
# covariates
Z = {t: data[data['date'] == t].drop(['yy', 'mm', 'date', 'ret'], axis=1).set_index('permno') for t in data['date'].unique()}
# returns
R = {t: data[data['date'] == t].ret for t in data['date'].unique()}

In [70]:
# number of latent factors
K = 3

# Read Fama-French factors as pre-specifed factors
gFac = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv')
gFac = gFac.query("199001 <= year_month <= 201405").drop('year_month', axis=1).T
gFac.columns = R.keys()

In [71]:
# matrix left/right division (following MATLAB function naming)
_mldivide = lambda denom, numer: sla.lstsq(np.array(denom), np.array(numer))[0]
_mrdivide = lambda numer, denom: (sla.lstsq(np.array(denom).T, np.array(numer).T)[0]).T

def _sign_convention(gamma, fac):
    '''
    sign the latent factors to have positive mean, and sign gamma accordingly
    '''
    sign_conv = fac.mean(axis=1).apply(lambda x: 1 if x >= 0 else -1)
    return gamma.mul(sign_conv.values, axis=1), fac.mul(sign_conv.values, axis=0)

def _calc_r2(r_act, r_fit):
    '''
    compute r2 of fitted values vs actual
    '''
    sumsq = lambda x: x.dot(x)
    sse = sum(sumsq(r_act[t].values - r_fit[t].values) for t in r_fit.keys())
    sst = sum(sumsq(r_act[t].values) for t in r_fit.keys())
    return 1. - sse / sst

In [72]:
# time periods and characteristics
times, charas = sorted(list(Z.keys())), Z[list(Z.keys())[0]].columns

# gLambda is the mean of observed factors
gLambda = gFac.mean(axis=1)

# index for latent and observable factors
fIdx, gIdx = list(map(str, range(1, K+1))), list(gFac.index)

# number of latent factors, observable factors, characteristics, and time periods
K, M, L, T = K, len(gIdx), len(charas), len(times)

In [73]:
# transform inputs
N_valid = pd.Series(index=times)

# characteristics sorted portfolio
X = pd.DataFrame(index=charas, columns=times)
# second moments of characteristics
W = {t: pd.DataFrame(index=charas, columns=charas) for t in times}

for t in times:
    is_valid = pd.DataFrame({'z':Z[t].notnull().all(axis=1),'r':R[t].notnull()}).all(axis=1) # not valid if ret or any charas are missing
    z_valid = Z[t].loc[is_valid,:]
    r_valid = R[t].loc[is_valid].values
    N_valid[t] = (1. * is_valid).sum()

    X[t] = z_valid.T.dot(r_valid) / N_valid[t]
    W[t] = z_valid.T.dot(z_valid) / N_valid[t]

In [74]:
# initial guess
Gamma0 = GammaDelta0 = pd.DataFrame(0., index=charas, columns=gIdx)

svU, svS, svV = ssla.svds(X.values, K) # careful that svds returns an ascending order of singular values different from svd
svU, svS, svV = np.fliplr(svU), svS[::-1], np.flipud(svV) # reverse order

fFac0 = pd.DataFrame(np.diag(svS).dot(svV), index=fIdx, columns=times) # first K PC of X
GammaBeta0 = pd.DataFrame(svU, index=charas, columns=fIdx) # first K eigvec of X
GammaBeta0, fFac0 = _sign_convention(GammaBeta0, fFac0)
Gamma0 = pd.concat([GammaBeta0, GammaDelta0], axis=1)

In [75]:
def _ipca_als_estimation(Gamma0):
    '''
    Runs one iteration of the alternating least squares estimation process
    [Inputs]
    Gamma0 (df(Lx(K+M))): previous iteration's Gamma estimate

    [Outputs]
    Gamma1 (df(Lx(K+M))): current iteration's Gamma estimate
    fFac1 (df(KxT)): current iteration's latent Factor estimate
    
    * Imposes identification assumption on Gamma1 and fFac1:
      Gamma1 is orthonormal matrix and fFac1 orthogonal with positive mean (taken across times)
    '''

    # 1. estimate latent factor
    fFac1 = pd.DataFrame(index=fIdx, columns=times)
    GammaBeta0, GammaDelta0 = Gamma0[fIdx], Gamma0[gIdx]
    for t in times:
        numer = GammaBeta0.T.dot(X[t])
        numer -= GammaBeta0.T.dot(W[t]).dot(GammaDelta0).dot(gFac[t])
        denom = GammaBeta0.T.dot(W[t]).dot(GammaBeta0)
        fFac1[t] = pd.Series(_mldivide(denom, numer), index=fIdx)

    # 2. estimate gamma
    vec_len = L * (K + M)
    numer, denom = np.zeros(vec_len), np.zeros((vec_len, vec_len))
    for t in times:
        Fac = pd.concat([fFac1[t], gFac[t]])
        FacOutProd = np.outer(Fac, Fac)
        numer += np.kron(X[t], Fac) * N_valid[t]
        denom += np.kron(W[t], FacOutProd) * N_valid[t] # this line takes most of the time
    Gamma1_tmp = np.reshape(_mldivide(denom, numer), (L, K + M))
    Gamma1 = pd.DataFrame(Gamma1_tmp, index=charas, columns=fIdx + gIdx)

    # 3. identification assumption
    # GammaBeta orthonormal and fFac1 orthogonal
    GammaBeta1, GammaDelta1 = Gamma1[fIdx], Gamma1[gIdx]
    R1 = sla.cholesky(GammaBeta1.T.dot(GammaBeta1))
    R2, _, _ = sla.svd(R1.dot(fFac1).dot(fFac1.T).dot(R1.T))
    GammaBeta1 = pd.DataFrame(_mrdivide(GammaBeta1, R1).dot(R2), index=charas, columns=fIdx)
    fFac1 = pd.DataFrame(_mldivide(R2, R1.dot(fFac1)), index=fIdx, columns=times)
    GammaBeta1, fFac1 = _sign_convention(GammaBeta1, fFac1)
    
    # orthogonality between GammaBeta and GammaDelta
    GammaDelta1 = (np.identity(L) - GammaBeta1.dot(GammaBeta1.T)).dot(GammaDelta1)
    fFac1 += GammaBeta1.T.dot(GammaDelta1).dot(gFac) # (K x M reg coef) * gFac
    GammaBeta1, fFac1 = _sign_convention(GammaBeta1, fFac1)
    Gamma1 = pd.concat([GammaBeta1, GammaDelta1], axis=1)
    return Gamma1, fFac1

In [76]:
# ALS estimate
MaxIter, MinTol = 100, 1e-6
tol, iter = float('inf'), 0

while iter < MaxIter and tol > MinTol:
    Gamma1, fFac1 = _ipca_als_estimation(Gamma0)
    tol_Gamma = abs(Gamma1 - Gamma0).values.max()
    tol_fFac = abs(fFac1 - fFac0).values.max()
    tol = max(tol_Gamma, tol_fFac)
    print('iter {}: tolGamma = {} and tolFac = {}'.format(iter, tol_Gamma, tol_fFac))
    Gamma0, fFac0 = Gamma1, fFac1
    iter += 1

iter 0: tolGamma = 1.4743221878840203 and tolFac = 355423.22839059203
iter 1: tolGamma = 0.9417595373067612 and tolFac = 2.5153631169824454e-08
iter 2: tolGamma = 1.2699515278248215 and tolFac = 2.3849750508483754e-06
iter 3: tolGamma = 1.3808879805838146 and tolFac = 2.423834897511595e-06
iter 4: tolGamma = 0.9797708166131988 and tolFac = 2.775027812546112e-06
iter 5: tolGamma = 0.8717686637955396 and tolFac = 2.761118057764107e-06
iter 6: tolGamma = 1.2114239118746208 and tolFac = 3.0327256986030778e-06
iter 7: tolGamma = 1.1446892249273681 and tolFac = 3.013069354809602e-06
iter 8: tolGamma = 1.2462184504545875 and tolFac = 2.479394496589732e-05
iter 9: tolGamma = 1.0522472575375572 and tolFac = 2.3924332113291407e-05
iter 10: tolGamma = 1.2524579988097337 and tolFac = 1.0519790847568364e-06
iter 11: tolGamma = 1.2843531367643095 and tolFac = 4.021461104733508e-06
iter 12: tolGamma = 1.1296497424251164 and tolFac = 3.949025665257096e-06
iter 13: tolGamma = 1.8098039119477831 and tol

In [89]:
fFac = pd.concat([fFac1, gFac], axis=0)

In [90]:
# store results
fitvals, r2 = {}, pd.Series()

fitvals['R_DRP'] = {t: Z[t].dot(Gamma1).dot(fFac[t].values) for t in times}
r2['R_Tot'] = _calc_r2(R, fitvals['R_DRP'])

In [97]:
R

{'1990-01-31': 211        0.000000
 519       -0.162791
 958        0.061538
 1105       0.555556
 1186      -0.127273
              ...   
 1402913   -0.129032
 1403089    0.012821
 1403111   -0.232558
 1403357    0.600000
 1403613   -0.020833
 Name: ret, Length: 2744, dtype: float64,
 '1990-02-28': 212       -0.250000
 520        0.277778
 959        0.048913
 1106       0.107143
 1187       0.041667
              ...   
 1402110    0.360000
 1402914   -0.009259
 1403112   -0.151515
 1403358    0.062500
 1403614   -0.063830
 Name: ret, Length: 2743, dtype: float64,
 '1990-03-31': 213        0.000000
 521        0.000000
 960       -0.010417
 1107       0.129032
 1188       0.080000
              ...   
 1402111    0.352941
 1402915    0.280374
 1403113    0.303571
 1403359    0.235294
 1403615   -0.022727
 Name: ret, Length: 2752, dtype: float64,
 '1990-04-30': 214        0.000000
 522       -0.043478
 961       -0.042105
 1108       0.085714
 1189      -0.018519
              ...   